<a href="https://colab.research.google.com/github/kavyajeetbora/geemap/blob/master/notebooks/02_geemap_using_basemaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [7]:
import ee
import geemap

## Earth Engine Authentication and Initialization

In [ ]:
ee.Authenticate()

In [5]:
ee.Initialize()

## Create an interactive map

In [6]:
geemap.show_youtube('6J5ZCIUPXfI')

Output(layout=Layout(width='815px'))

## Using basemaps

By default the geemap loads the openstreet basemap

In [9]:
Map = geemap.Map(center = [40, -100], zoom=5)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

Basemaps in geemap

In [13]:
Map = geemap.Map(center = [40, -100], zoom=5)
Map.add_basemap("ROADMAP")
Map.add_basemap("Esri National Geographic")
Map

Basemap can only be one of the following:
  OpenStreetMap
  Esri.WorldStreetMap
  Esri.WorldImagery
  Esri.WorldTopoMap
  FWS NWI Wetlands
  FWS NWI Wetlands Raster
  NLCD 2021 CONUS Land Cover
  NLCD 2019 CONUS Land Cover
  NLCD 2016 CONUS Land Cover
  NLCD 2013 CONUS Land Cover
  NLCD 2011 CONUS Land Cover
  NLCD 2008 CONUS Land Cover
  NLCD 2006 CONUS Land Cover
  NLCD 2004 CONUS Land Cover
  NLCD 2001 CONUS Land Cover
  USGS NAIP Imagery
  USGS NAIP Imagery False Color
  USGS NAIP Imagery NDVI
  USGS Hydrography
  USGS 3DEP Elevation
  ESA Worldcover 2020
  ESA Worldcover 2020 S2 FCC
  ESA Worldcover 2020 S2 TCC
  ESA Worldcover 2021
  ESA Worldcover 2021 S2 FCC
  ESA Worldcover 2021 S2 TCC
  BasemapAT.basemap
  BasemapAT.grau
  BasemapAT.highdpi
  BasemapAT.orthofoto
  BasemapAT.overlay
  BasemapAT.surface
  BasemapAT.terrain
  CartoDB.DarkMatter
  CartoDB.DarkMatterNoLabels
  CartoDB.DarkMatterOnlyLabels
  CartoDB.Positron
  CartoDB.PositronNoLabels
  CartoDB.PositronOnlyLabels
 

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

The full basemap list available in geemap can be found in the source code: [basemap.py](https://github.com/gee-community/geemap/blob/master/geemap/basemaps.py)

More WMS (Web Mapping Services) basemaps can be found at the following websites:

  1. USGS National Map: https://viewer.nationalmap.gov/services/
  2. MRLC NLCD Land Cover data: https://viewer.nationalmap.gov/services/
  3. FWS NWI Wetlands data: https://www.fws.gov/wetlands/Data/Web-Map-Services.html


In [18]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap('ROADMAP')
naip_url = r"https://imagery.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?"
Map.add_wms_layer(
    url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…